In [36]:
from pororo import Pororo
from pororo.tasks.utils.download_utils import download_or_load
from pororo.tasks import PororoAsrFactory
import torch
import fairseq
from fairseq.tasks.audio_pretraining import AudioPretrainingTask
import soundfile as sf
import torch.nn.functional as F


In [40]:
model_path = 'misc/wav2vec.ko.pt'

model = torch.load(model_path)
w2v_args = model['pretrain_args']
task = AudioPretrainingTask.setup_task(w2v_args)
w2v_model = task.build_model(w2v_args).cuda()
print(w2v_args)

Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9,0.98)', adam_eps=1e-06, all_gather_list_size=16384, arch='wav2vec2', attention_dropout=0.1, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_suffix='', clip_norm=0.0, codebook_negatives=0, conv_bias=False, conv_feature_layers='[(512, 10, 5)] + [(512, 3, 2)] * 4 + [(512,2,2)] * 2', conv_pos=128, conv_pos_groups=16, cpu=False, criterion='wav2vec', cross_sample_negatives=0, curriculum=0, data='./manifest/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method='env://', distributed_no_spawn=True, distributed_port=-1, distributed_rank=0, distributed_world_size=64, distributed_wrapper='DDP', dropout=0.0, dropout_features=0.1, dropout_input=0.1, empty_cache_freq=0, enable_padding=False, encoder_attention_heads=16, encoder_embed_dim=1024, encoder_ffn_embed_dim=4096, en

In [46]:
audio_file = '/mnt/data_input/speechDATA_eng/train-clean-100/103/1240/103-1240-0000.flac'
x, sr = sf.read(audio_file)

print(len(x))
source = torch.from_numpy(x).float().cuda()
# normalize
source = F.layer_norm(source, source.shape)

# (len(data),) -> (1,-1)
source = source.view(1, -1)


input_args = {
    "source": source,
    "padding_mask": None,
    "mask": False,
}
w2v_feat, _ = w2v_model.extract_features(**input_args)
print(f"expected feature len: {(len(x)//16000) * 50}")
print(w2v_feat.shape)

225360
expected feature len: 14.085
torch.Size([1, 704, 1024])
